# 필요과정

In [206]:
import numpy as np 
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import functions as myfunc
import warnings
warnings.filterwarnings(action='ignore')

from sdv.tabular import CTGAN 

def save_pickle(data_name, data):
    with open(data_name, 'wb') as f:
        pickle.dump(data, f)
        
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/HANBatang.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)        

rawdata = pd.read_csv("2016_housemasterdata.csv" , encoding = 'CP949')

main_variable = ["MD제공용_가구고유번호", "수도권여부" , "가구주_성별코드", "가구주_교육정도_학력코드" , "가구주_만연령" , "가구원수","가구주_혼인상태코드" , "가구주_종사상지위코드", '경상소득(조사)', "경상소득_근로소득(조사)" , "경상소득_사업소득(조사)", "경상소득_재산소득(조사)" ,"경상소득_이전소득(조사)[공적이전소득+사적이전소득]" , "지출(조사)[비소비지출(조사)+소비지출(조사)](2012~2018년: 복지, 2019년~:공통)" ,
                 "지출_소비지출", "지출_소비지출_식료품(외식비포함)" ,"지출_소비지출_주거비", "지출_소비지출_교육비(보육료포함)", "지출_소비지출_의료비", "지출_소비지출_교통비", "지출_소비지출_통신비", "지출_소비지출_경조비및기타소비지출(2016년까지)" ,
                 "지출_비소비지출(조사)" , "지출_비소비지출_세금(조사)" , "지출_비소비지출_공적연금사회보험료(조사)" , "지출_비소비지출_가구간이전지출(조사)" , "지출_비소비지출_비영리단체이전지출(조사)" , "지출_비소비지출_연간지급이자(조사)"]

main_variable_ko = ["id", "수도권여부" , "가구주_성별코드", "교육정도", "만연령","가구원수" ,"혼인상태" ,
                    "가구주종사직업","총소득", "근로소득" , "사업소득", "재산소득" , "이전소득" , "총지출" ,
                    "소비지출", "식료품", "주거비", "교육비", "의료비", "교통비", "통신비", 
                    "기타소비" , "비소비지출" ,"세금" , "공적연금사회보험료" , "가구간이전지출" , "비영리단체이전지출" , "연간지급이자"]

main_variable_en = ["id", "metro", "sex", "edu", "age", "family_number" , "marital status" , 
                    "housemaster_job" , "income", "income_lab", "income_bis", "income_property", "income_trans", "all_consump" ,
                    "consump", "consump_food", "consump_house", "consump_edu", "consump_medi", "consump_transp", "consump_comm", 
                    "consump_others" , "nonconsump" ,"nonconsump_tax" , "nonconsump_pension" , "nonconsump_transfer" , "nonconsump_organization" , "nonconsump_interest"]

main_variable_en_consump = ["all_consump" ,"consump", "consump_food", "consump_house", "consump_edu", "consump_medi", 
                            "consump_transp", "consump_comm", "consump_others"]

dict_var = myfunc.make_dict(main_variable_en, main_variable_ko)
dict_sex = {'1':'M', '2':'F'}
dict_edu = {'1':'안받음', '2':'초등', '3': '중등', '4': '고등', '5':'대학', '6':'대학교', '7':'대학원'}
dict_marry = {'1' : '미혼' , '2' : '배우자있음' , '3' : '사별' , '4': '이혼'}
dict_housejob = {'1' : '상용근로자', '2' : '임시.일용근로자', '3' : '고용원이 있는 자영업자', '4' : '고용원이 없는 자영업자', '5' : '무급가족종사자', '6' : '기타 종사자(실적급의 보험설계사, 대리 운전기사, 학습지 방문 교사 등)', '7' : '기타(무직자, 가사, 학생 등)'}

dict_familynumber = {}
for i in range(1, 10):
    dict_familynumber[str(i)] = f'{i}명' # 범주형으로 표현하기 위해

all_oridata = rawdata.loc[rawdata['조사부문코드'] == "P", [ '조사부문코드'] + main_variable] # 복지부문 , 금융부문에서 복지부분만 추출하기 위해 
all_oridata.drop(columns=['조사부문코드'], inplace=True)
all_oridata.columns = main_variable_en

final_data = myfunc.convert_int(all_oridata, main_variable_en_consump) # 데이터 int형식으로 바꾸기 
final_data = myfunc.convert_int_to_str(final_data, ['sex', 'edu' ,'marital status', 'housemaster_job' , 'family_number']) # 해당 범주는 str형식으로 바꾸기 

myfunc.change_contents(final_data, 'sex', dict_sex)
myfunc.change_contents(final_data, 'edu', dict_edu)
myfunc.change_contents(final_data, 'marital status', dict_marry)
myfunc.change_contents(final_data, 'housemaster_job', dict_housejob)
myfunc.change_contents(final_data, 'family_number', dict_familynumber)

# final_data['metro'] = final_data['metro'].str.replace('G','')

,id,metro,sex,edu,age,family_number,marital status,housemaster_job,income,income_lab,...,consump_medi,consump_transp,consump_comm,consump_others,nonconsump,nonconsump_tax,nonconsump_pension,nonconsump_transfer,nonconsump_organization,nonconsump_interest
2,10200151,G1,F,대학교,53,3명,배우자있음,상용근로자,12600,12600,...,200,430,310,1300,1896,981,725,0,0,190
3,10200161,G1,M,대학,46,5명,배우자있음,고용원이 있는 자영업자,4560,0,...,36,120,240,380,1589,86,636,0,0,867
4,10200171,G1,M,대학교,36,2명,배우자있음,상용근로자,12420,12420,...,35,700,120,800,2759,515,818,200,136,1090
5,10200191,G1,F,안받음,80,1명,사별,임시.일용근로자,612,160,...,92,10,36,40,9,1,0,0,8,0
6,10200211,G2,M,대학,57,4명,배우자있음,고용원이 없는 자영업자,10800,0,...,100,36,480,230,1137,161,756,0,150,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18249,991600301,G2,M,안받음,86,1명,사별,"기타(무직자, 가사, 학생 등)",240,0,...,50,30,30,40,1,1,0,0,0,0
18269,994200381,G2,F,안받음,84,1명,사별,"기타(무직자, 가사, 학생 등)",288,0,...,30,20,10,10,11,1,10,0,0,0
18270,994505311,G2,M,대학,41,5명,배우자있음,상용근로자,4540,4500,...,200,400,240,500,593,82,331,0,0,180
18271,994505321,G2,M,초등,54,4명,배우자있음,상용근로자,2160,2160,...,0,100,72,300,1,1,0,0,0,0


# 방법1 그냥 재현하기

In [227]:
model = CTGAN(epochs = 600 , batch_size = 300 , verbose = True)
model.fit(final_data)
synthetic_data1 = model.sample(len(final_data))

Epoch 1, Loss G:  0.3864,Loss D: -0.7369
Epoch 2, Loss G: -0.3700,Loss D:  0.1714
Epoch 3, Loss G:  0.2278,Loss D:  0.1181
Epoch 4, Loss G: -0.9063,Loss D:  0.4060
Epoch 5, Loss G: -1.7097,Loss D:  0.4015
Epoch 6, Loss G: -1.3817,Loss D:  0.5506
Epoch 7, Loss G: -1.3893,Loss D:  0.0900
Epoch 8, Loss G: -1.7527,Loss D:  0.0169
Epoch 9, Loss G: -1.7637,Loss D: -0.0055
Epoch 10, Loss G: -2.0128,Loss D: -0.3239
Epoch 11, Loss G: -2.2653,Loss D: -0.0699
Epoch 12, Loss G: -2.1726,Loss D:  0.2174
Epoch 13, Loss G: -2.6646,Loss D:  0.0210
Epoch 14, Loss G: -2.7395,Loss D:  0.1151
Epoch 15, Loss G: -3.0011,Loss D: -0.0466
Epoch 16, Loss G: -2.7327,Loss D:  0.0724
Epoch 17, Loss G: -2.8796,Loss D:  0.0506
Epoch 18, Loss G: -3.2749,Loss D: -0.1702
Epoch 19, Loss G: -3.1102,Loss D:  0.0391
Epoch 20, Loss G: -2.8759,Loss D: -0.2882
Epoch 21, Loss G: -3.3734,Loss D: -0.1890
Epoch 22, Loss G: -3.0025,Loss D: -0.2466
Epoch 23, Loss G: -3.0958,Loss D:  0.0514
Epoch 24, Loss G: -3.2991,Loss D: -0.3516
E

In [228]:
synthetic_data1.to_csv("scenario1.csv" , index= False)

# 방법2 그룹화 재현하기

0이 제일 적은 변수들 그룹화

In [210]:
cond1 = [final_data['income_lab'] == 0 , final_data['income_lab'] > 0]
cond2 = [final_data['consump_food'] == 0 , final_data['consump_food'] > 0] 
cond3 = [final_data['nonconsump_tax'] == 0 , final_data['nonconsump_tax'] > 0]

less_data_list = []

for k in range(len(cond1)):
    for j in range(len(cond2)):
        for i in range(len(cond3)):
            aa = final_data[cond1[k] & cond2[j] & cond3[i]]
            less_data_list.append(aa)


In [211]:
[len(less_data_list[i]) for i in range(0,8)]

[1, 2, 199, 2727, 1, 3, 90, 6074]

In [215]:
model1 = CTGAN(epochs = 600 , batch_size = 10 , verbose = True) # 10 ~ 100개
model2 = CTGAN(epochs = 600 , batch_size = 100 , verbose = True) # 100 ~ 1000개 
model3 = CTGAN(epochs = 600 , batch_size = 300 , verbose = True) # 1000 ~ 3000개 
model4 = CTGAN(epochs = 600 , batch_size = 500 , verbose = True) # 3000개 이상 

In [216]:
less_list_df = []
for i in range(0,8):
    less_list_df.append("")

In [217]:
for i in range(0,8):
    if len(less_data_list[i]) < 10  :
        less_list_df[i] = less_data_list[i]
        
    elif 10 <= len(less_data_list[i]) <= 100 :   
        model1.fit(less_data_list[i])
        less_list_df[i] = model1.sample(len(less_data_list[i]))
        
    elif 100 <= len(less_data_list[i]) <= 1000 : 
        model2.fit(less_data_list[i])
        less_list_df[i] = model2.sample(len(less_data_list[i]))
        
    elif 1000 <= len(less_data_list[i]) < 3000 : 
        model3.fit(less_data_list[i])
        less_list_df[i] = model3.sample(len(less_data_list[i]))                
        
    else : 
        model4.fit(less_data_list[i])
        less_list_df[i] = model4.sample(len(less_data_list[i])) 

Epoch 1, Loss G:  1.1932,Loss D: -0.0030
Epoch 2, Loss G:  1.2464,Loss D: -0.0533
Epoch 3, Loss G:  1.2912,Loss D: -0.1713
Epoch 4, Loss G:  1.2372,Loss D: -0.1311
Epoch 5, Loss G:  1.4816,Loss D: -0.1677
Epoch 6, Loss G:  1.3473,Loss D: -0.2344
Epoch 7, Loss G:  1.3296,Loss D: -0.3692
Epoch 8, Loss G:  1.3491,Loss D: -0.4068
Epoch 9, Loss G:  1.4657,Loss D: -0.4893
Epoch 10, Loss G:  1.3170,Loss D: -0.3756
Epoch 11, Loss G:  1.3061,Loss D: -0.5754
Epoch 12, Loss G:  1.3169,Loss D: -0.6766
Epoch 13, Loss G:  1.1830,Loss D: -0.7643
Epoch 14, Loss G:  1.3012,Loss D: -0.7752
Epoch 15, Loss G:  1.1214,Loss D: -0.9485
Epoch 16, Loss G:  1.2693,Loss D: -1.1682
Epoch 17, Loss G:  1.3224,Loss D: -0.8353
Epoch 18, Loss G:  1.1007,Loss D: -1.2638
Epoch 19, Loss G:  1.1115,Loss D: -1.4686
Epoch 20, Loss G:  1.1860,Loss D: -1.3474
Epoch 21, Loss G:  1.0504,Loss D: -1.8446
Epoch 22, Loss G:  1.1970,Loss D: -1.3503
Epoch 23, Loss G:  0.9954,Loss D: -1.8618
Epoch 24, Loss G:  0.9417,Loss D: -1.7265
E

0이 많은 변수들 그룹화

In [218]:
syn_file_names = ["less_syn_data_scenario2" + str(i) + ".csv" for i in range(0, 8)]
for j in range(0,8):
    less_list_df[j].to_csv(syn_file_names[j], index = False)

In [219]:
synthetic_data = pd.concat([less_list_df[i] for i in range(0,8)] , axis = 0) 

synthetic_data.to_csv("scenario2_less_syn_data.csv" , index = False)

# 방법3 그룹화 데이터 rdt이용해서 ctgan 재현하기

첫번째 그룹화( income_lab )

In [235]:
less_transformed_data = []

for i in range(0, 8):
    file_name = "less_transformed_data" + str(i) + ".csv"
    df = pd.read_csv(file_name)
    less_transformed_data.append(df)

In [236]:
less_syn_list = []
for i in range(0,8):
    less_syn_list.append("")

In [237]:
model1 = CTGAN(epochs = 600 , batch_size = 10 , verbose = True) # 10 ~ 100개
model2 = CTGAN(epochs = 600 , batch_size = 100 , verbose = True) # 100 ~ 1000개 
model3 = CTGAN(epochs = 600 , batch_size = 300 , verbose = True) # 1000 ~ 3000개 
model4 = CTGAN(epochs = 600 , batch_size = 500 , verbose = True) # 3000개 이상 

In [238]:
for i in range(0,8):
    if len(less_transformed_data[i]) < 10  :
        less_syn_list[i] = less_transformed_data[i]
        
    elif 10 <= len(less_transformed_data[i]) <= 100 :   
        model1.fit(less_transformed_data[i])
        less_syn_list[i] = model1.sample(len(less_transformed_data[i]))
        
    elif 100 <= len(less_transformed_data[i]) <= 1000 : 
        model2.fit(less_transformed_data[i])
        less_syn_list[i] = model2.sample(len(less_transformed_data[i]))
        
    elif 1000 <= len(less_transformed_data[i]) < 3000 : 
        model3.fit(less_transformed_data[i])
        less_syn_list[i] = model3.sample(len(less_transformed_data[i]))                
        
    else : 
        model4.fit(less_transformed_data[i])
        less_syn_list[i] = model4.sample(len(less_transformed_data[i])) 

Epoch 1, Loss G:  0.0114,Loss D: -0.0381
Epoch 2, Loss G: -0.0090,Loss D: -0.0063
Epoch 3, Loss G:  0.0156,Loss D: -0.0606
Epoch 4, Loss G:  0.0109,Loss D: -0.0683
Epoch 5, Loss G:  0.0331,Loss D: -0.1045
Epoch 6, Loss G:  0.0482,Loss D: -0.1754
Epoch 7, Loss G: -0.0634,Loss D: -0.2005
Epoch 8, Loss G:  0.0258,Loss D: -0.1629
Epoch 9, Loss G:  0.0304,Loss D: -0.3048
Epoch 10, Loss G:  0.0694,Loss D: -0.2841
Epoch 11, Loss G:  0.0001,Loss D: -0.4631
Epoch 12, Loss G: -0.0443,Loss D: -0.4003
Epoch 13, Loss G:  0.0039,Loss D: -0.4819
Epoch 14, Loss G: -0.1705,Loss D: -0.5603
Epoch 15, Loss G: -0.1848,Loss D: -0.5165
Epoch 16, Loss G: -0.1724,Loss D: -0.5398
Epoch 17, Loss G: -0.1960,Loss D: -0.6005
Epoch 18, Loss G: -0.2015,Loss D: -0.6743
Epoch 19, Loss G: -0.2282,Loss D: -0.7182
Epoch 20, Loss G: -0.4294,Loss D: -0.4804
Epoch 21, Loss G: -0.5238,Loss D: -1.0628
Epoch 22, Loss G: -0.7708,Loss D: -0.7988
Epoch 23, Loss G: -0.6098,Loss D: -0.6253
Epoch 24, Loss G: -0.9325,Loss D: -0.7529
E

In [239]:
less_syn_transform_file_names = ["less_syn_transform_data_group_" + str(i) + ".csv" for i in range(0, 8)]
for j in range(0,8):
    less_syn_list[j].to_csv(less_syn_transform_file_names[j], index = False)

두번째 그룹화 ( income_property )

In [240]:
more_transformed_data = []

for i in range(0, 8):
    file_name = "more_transformed_data" + str(i) + ".csv"
    df = pd.read_csv(file_name)
    more_transformed_data.append(df)

In [241]:
more_syn_list = []
for i in range(0,8):
    more_syn_list.append("")

In [242]:
model1 = CTGAN(epochs = 600 , batch_size = 10 , verbose = True) # 10 ~ 100개
model2 = CTGAN(epochs = 600 , batch_size = 100 , verbose = True) # 100 ~ 1000개 
model3 = CTGAN(epochs = 600 , batch_size = 300 , verbose = True) # 1000 ~ 3000개 
model4 = CTGAN(epochs = 600 , batch_size = 500 , verbose = True) # 3000개 이상 

In [243]:
for i in range(0,8):
    if len(more_transformed_data[i]) < 10  :
        more_syn_list[i] = more_transformed_data[i]
        
    elif 10 <= len(more_transformed_data[i]) <= 100 :   
        model1.fit(more_transformed_data[i])
        more_syn_list[i] = model1.sample(len(more_transformed_data[i]))
        
    elif 100 <= len(more_transformed_data[i]) <= 1000 : 
        model2.fit(more_transformed_data[i])
        more_syn_list[i] = model2.sample(len(more_transformed_data[i]))
        
    elif 1000 <= len(more_transformed_data[i]) < 3000 : 
        model3.fit(more_transformed_data[i])
        more_syn_list[i] = model3.sample(len(more_transformed_data[i]))                
        
    else : 
        model4.fit(more_transformed_data[i])
        more_syn_list[i] = model4.sample(len(more_transformed_data[i])) 

Epoch 1, Loss G:  0.0815,Loss D: -0.0599
Epoch 2, Loss G: -0.0467,Loss D: -0.1789
Epoch 3, Loss G: -0.4110,Loss D: -0.2673
Epoch 4, Loss G: -0.7112,Loss D: -0.2299
Epoch 5, Loss G: -0.9057,Loss D:  0.0626
Epoch 6, Loss G: -0.8447,Loss D:  0.0591
Epoch 7, Loss G: -0.6799,Loss D: -0.2261
Epoch 8, Loss G: -0.5664,Loss D:  0.0862
Epoch 9, Loss G: -0.4467,Loss D:  0.0242
Epoch 10, Loss G: -0.4004,Loss D:  0.0190
Epoch 11, Loss G: -0.6123,Loss D:  0.1795
Epoch 12, Loss G: -0.7984,Loss D:  0.1998
Epoch 13, Loss G: -1.0281,Loss D:  0.4993
Epoch 14, Loss G: -0.9382,Loss D:  0.3077
Epoch 15, Loss G: -0.8989,Loss D: -0.0840
Epoch 16, Loss G: -1.0521,Loss D: -0.2167
Epoch 17, Loss G: -1.1644,Loss D:  0.0472
Epoch 18, Loss G: -1.2407,Loss D:  0.0287
Epoch 19, Loss G: -1.3601,Loss D:  0.2624
Epoch 20, Loss G: -1.4478,Loss D:  0.3449
Epoch 21, Loss G: -1.1857,Loss D:  0.2639
Epoch 22, Loss G: -0.9545,Loss D: -0.1147
Epoch 23, Loss G: -0.8024,Loss D: -0.2344
Epoch 24, Loss G: -0.7374,Loss D: -0.1381
E

In [244]:
more_syn_transform_file_names = ["more_syn_transform_data_group_" + str(i) + ".csv" for i in range(0, 8)]
for j in range(0,8):
    more_syn_list[j].to_csv(more_syn_transform_file_names[j], index = False)

# 방법3 그룹화 데이터 rdt이용해서 ctgan 재현하기 ( Frequency Encoder )

In [190]:
transformed_data = []

for i in range(0, 8):
    file_name = "transformed_data" + str(i) + ".csv"
    df = pd.read_csv(file_name)
    transformed_data.append(df)

In [191]:
syn_list = []
for i in range(0,8):
    syn_list.append("")

In [192]:
model1 = CTGAN(epochs = 600 , batch_size = 10 , verbose = True) # 10 ~ 100개
model2 = CTGAN(epochs = 600 , batch_size = 100 , verbose = True) # 100 ~ 1000개 
model3 = CTGAN(epochs = 600 , batch_size = 300 , verbose = True) # 1000 ~ 3000개 
model4 = CTGAN(epochs = 600 , batch_size = 500 , verbose = True) # 3000개 이상 

In [194]:
for i in range(0,8):
    if len(transformed_data[i]) < 10  :
        syn_list[i] = transformed_data[i]
        
    elif 10 <= len(transformed_data[i]) <= 100 :   
        model1.fit(transformed_data[i])
        syn_list[i] = model1.sample(len(transformed_data[i]))
        
    elif 100 <= len(transformed_data[i]) <= 1000 : 
        model2.fit(transformed_data[i])
        syn_list[i] = model2.sample(len(transformed_data[i]))
        
    elif 1000 <= len(transformed_data[i]) < 3000 : 
        model3.fit(transformed_data[i])
        syn_list[i] = model3.sample(len(transformed_data[i]))                
        
    else : 
        model4.fit(transformed_data[i])
        syn_list[i] = model4.sample(len(transformed_data[i])) 

Epoch 1, Loss G:  0.0517,Loss D: -0.0215
Epoch 2, Loss G:  0.0182,Loss D: -0.0438
Epoch 3, Loss G:  0.0002,Loss D: -0.2492
Epoch 4, Loss G:  0.0052,Loss D: -0.2427
Epoch 5, Loss G: -0.0973,Loss D: -0.3269
Epoch 6, Loss G: -0.0260,Loss D: -0.4305
Epoch 7, Loss G: -0.1756,Loss D: -0.3842
Epoch 8, Loss G: -0.2613,Loss D: -0.7644
Epoch 9, Loss G: -0.2246,Loss D: -0.6450
Epoch 10, Loss G: -0.4647,Loss D: -0.4663
Epoch 11, Loss G: -0.6482,Loss D: -0.6962
Epoch 12, Loss G: -0.6862,Loss D: -0.9348
Epoch 13, Loss G: -0.9049,Loss D: -1.0075
Epoch 14, Loss G: -1.0374,Loss D: -1.1673
Epoch 15, Loss G: -0.7708,Loss D: -0.9479
Epoch 16, Loss G: -1.2414,Loss D: -0.8664
Epoch 17, Loss G: -1.4259,Loss D: -0.5265
Epoch 18, Loss G: -1.7788,Loss D: -0.9024
Epoch 19, Loss G: -1.7609,Loss D: -0.1656
Epoch 20, Loss G: -1.7806,Loss D:  0.0848
Epoch 21, Loss G: -1.8945,Loss D: -0.0878
Epoch 22, Loss G: -1.7382,Loss D: -0.3841
Epoch 23, Loss G: -1.7062,Loss D: -0.2709
Epoch 24, Loss G: -1.9234,Loss D: -0.2353
E

In [195]:
syn_transform_file_names = ["frequency_transform_data_group_" + str(i) + ".csv" for i in range(0, 8)]
for j in range(0,8):
    syn_list[j].to_csv(syn_transform_file_names[j], index = False)

회귀모형 바꾼거

In [159]:
example_data = pd.read_csv("example_transform_data.csv")

In [160]:
example_model = CTGAN(epochs = 600 , batch_size = 400 , verbose = True)
example_model.fit(example_data)
syn_example = example_model.sample(len(example_data))

Epoch 1, Loss G:  0.7099,Loss D:  0.0642
Epoch 2, Loss G:  0.6394,Loss D:  0.0751
Epoch 3, Loss G:  0.6062,Loss D: -0.0431
Epoch 4, Loss G:  0.2616,Loss D:  0.0353
Epoch 5, Loss G: -0.0059,Loss D:  0.0228
Epoch 6, Loss G:  0.0886,Loss D: -0.0011
Epoch 7, Loss G: -0.1016,Loss D: -0.0942
Epoch 8, Loss G: -0.1546,Loss D:  0.1130
Epoch 9, Loss G: -0.1418,Loss D: -0.0878
Epoch 10, Loss G: -0.1939,Loss D:  0.0011
Epoch 11, Loss G: -0.1020,Loss D:  0.0187
Epoch 12, Loss G: -0.0586,Loss D: -0.0559
Epoch 13, Loss G: -0.0492,Loss D:  0.0024
Epoch 14, Loss G: -0.1297,Loss D: -0.0328
Epoch 15, Loss G: -0.1756,Loss D: -0.1648
Epoch 16, Loss G: -0.2070,Loss D:  0.0837
Epoch 17, Loss G: -0.1508,Loss D: -0.1449
Epoch 18, Loss G: -0.2064,Loss D: -0.1159
Epoch 19, Loss G: -0.3255,Loss D: -0.1278
Epoch 20, Loss G: -0.2722,Loss D: -0.1047
Epoch 21, Loss G: -0.3199,Loss D: -0.1700
Epoch 22, Loss G: -0.3494,Loss D: -0.1587
Epoch 23, Loss G: -0.3799,Loss D: -0.0029
Epoch 24, Loss G: -0.4198,Loss D: -0.1463
E

In [161]:
syn_example.to_csv("syn_example.csv" , index = False)

# Copula로 적합

In [180]:
from sdv.tabular import CopulaGAN

In [182]:
field_dist = {
                            "income": 'gaussian',
                            "income_lab": 'gaussian',
                            "income_bis": 'gaussian',
                            "income_property": 'gaussian',
                            "income_trans": 'gaussian',
                            "all_consump": 'gaussian',
                            "consump": 'gaussian',
                            "consump_food": 'gaussian',
                            "consump_house": 'gaussian',
                            "consump_edu": 'gaussian',
                            "consump_medi": 'gaussian',
                            "consump_transp": 'gaussian',
                            "consump_comm": 'gaussian',
                            "consump_others": 'gaussian',
                            "nonconsump": 'gaussian',
                            "nonconsump_tax": 'gaussian',
                            "nonconsump_pension": 'gaussian',
                            "nonconsump_transfer": 'gaussian',
                            "nonconsump_organization": 'gaussian',
                            "nonconsump_interest": 'gaussian'}

In [186]:
Copula_model1 = CopulaGAN(epochs = 600 , batch_size = 10 , verbose = True , 
                          field_distributions = field_dist)
                          

Copula_model2 = CopulaGAN(epochs = 600 , batch_size = 100 , verbose = True , field_distributions = field_dist)
Copula_model3 = CopulaGAN(epochs = 600 , batch_size = 300 , verbose = True , field_distributions = field_dist)
Copula_model4 = CopulaGAN(epochs = 600 , batch_size = 500 , verbose = True , field_distributions = field_dist)  


In [187]:
syn_list = []
for i in range(0,8):
    syn_list.append("")

In [188]:
for i in range(0,8):
    if len(transformed_data[i]) < 10  :
        syn_list[i] = transformed_data[i]
        
    elif 10 <= len(transformed_data[i]) <= 100 :   
        Copula_model1.fit(transformed_data[i])
        syn_list[i] = Copula_model1.sample(len(transformed_data[i]))
        
    elif 100 <= len(transformed_data[i]) <= 1000 : 
        Copula_model2.fit(transformed_data[i])
        syn_list[i] = Copula_model2.sample(len(transformed_data[i]))
        
    elif 1000 <= len(transformed_data[i]) < 3000 : 
        Copula_model3.fit(transformed_data[i])
        syn_list[i] = Copula_model3.sample(len(transformed_data[i]))                
        
    else : 
        Copula_model4.fit(transformed_data[i])
        syn_list[i] = Copula_model4.sample(len(transformed_data[i])) 

Epoch 1, Loss G:  0.0033,Loss D:  0.2653
Epoch 2, Loss G:  0.2688,Loss D: -0.3075
Epoch 3, Loss G:  0.1707,Loss D:  0.1534
Epoch 4, Loss G:  0.0300,Loss D:  0.1873
Epoch 5, Loss G:  0.2218,Loss D: -0.0852
Epoch 6, Loss G:  0.2083,Loss D:  0.0208
Epoch 7, Loss G:  0.1008,Loss D: -0.0112
Epoch 8, Loss G:  0.0779,Loss D:  0.3177
Epoch 9, Loss G:  0.0604,Loss D: -0.3094
Epoch 10, Loss G:  0.0336,Loss D:  0.1847
Epoch 11, Loss G:  0.2712,Loss D:  0.2479
Epoch 12, Loss G:  0.1809,Loss D:  0.1863
Epoch 13, Loss G:  0.0384,Loss D: -0.1892
Epoch 14, Loss G:  0.0982,Loss D: -0.1241
Epoch 15, Loss G:  0.0837,Loss D: -0.3209
Epoch 16, Loss G: -0.0366,Loss D:  0.0364
Epoch 17, Loss G: -0.0940,Loss D:  0.0138
Epoch 18, Loss G:  0.2755,Loss D:  0.1192
Epoch 19, Loss G:  0.2179,Loss D:  0.0637
Epoch 20, Loss G:  0.0958,Loss D: -0.0579
Epoch 21, Loss G:  0.2477,Loss D:  0.2127
Epoch 22, Loss G:  0.1342,Loss D: -0.2333
Epoch 23, Loss G:  0.1186,Loss D:  0.0568
Epoch 24, Loss G:  0.0986,Loss D: -0.1949
E

In [189]:
syn_transform_file_names = ["copula_syn_transform_data_group_" + str(i) + ".csv" for i in range(0, 8)]
for j in range(0,8):
    syn_list[j].to_csv(syn_transform_file_names[j], index = False)

## ㅇㅇ